Classic from 2009. See [GF tutorial](https://www.grammaticalframework.org/doc/tutorial/gf-tutorial.html#toc16) and several concrete syntaxes at [gf-contrib](https://github.com/GrammaticalFramework/gf-contrib/tree/master/foods).

In [1]:
-- (c) 2009 Aarne Ranta under LGPL

abstract Food = {
  flags startcat = Comment ;
  cat
    Comment ; Item ; Kind ; Quality ;
  fun
    Pred : Item -> Quality -> Comment ;       -- Predication: "item is quality"
    This, That, These, Those : Kind -> Item ; -- Quantification: turn the Kind ("pizza") into Item ("this pizza")
    Mod : Quality -> Kind -> Kind ;           -- Modification: "pizza" to "boring pizza"
    Wine, Cheese, Fish, Pizza : Kind ;        -- list of Kinds
    Very : Quality -> Quality ;               -- Enhance a Quality: "boring" to "very boring"
    Fresh, Warm, Italian, Boring,
         Expensive, Delicious : Quality ;     -- list of Qualities
}

Defined Food

In [2]:
gr

Pred (That Pizza) Italian


### English concrete syntax from scratch

We define our own inflection tables and agreement rules.

In [3]:
concrete FoodEng of Food = {
  
    lincat
      Comment, Quality = {s : Str} ;
      Kind = Noun ;       -- Noun defined later in this file.
      Item = NounPhrase ; -- NounPhrase defined later in this file.
  
    lin
      -- Syntactic functions that take arguments
      Pred item quality = {s = item.s ++ copula item.n ++ quality.s} ;
      This kind  = det Sg "this" kind ;
      That kind  = det Sg "that" kind ;
      These kind = det Pl "these" kind ;
      Those kind = det Pl "those" kind ;
      Mod quality kind = {s = table {n => quality.s ++ kind.s ! n}} ;

      -- Lexical functions, no arguments
      Wine   = regNoun "wine" ;
      Cheese = regNoun "cheese" ;
      Fish   = noun "fish" "fish" ;
      Pizza  = regNoun "pizza" ;
      Very qual = {s = "very" ++ qual.s} ;
      Fresh     = adj "fresh" ;
      Warm      = adj "warm" ;
      Italian   = adj "Italian" ;
      Expensive = adj "expensive" ;
      Delicious = adj "delicious" ;
      Boring    = adj "boring" ;

    param
      Number = Sg | Pl ;
  
    oper
      -- Defining our own record types.
      -- They are used in the lincats.
      Noun : Type = {s : Number => Str} ;
      NounPhrase : Type = {s : Str ; n : Number} ;
    
      -- Makes Noun into a Noun phrase.
      det : Number -> Str -> Noun -> NounPhrase =
        \n,d,cn -> {
          s = d ++ cn.s ! n ;
          n = n
        } ;
    
      -- Constructs a noun from singular and plural.
      noun : Str -> Str -> Noun =
        \man,men -> {s = table {
          Sg => man ;
          Pl => men
          }
        } ;
      -- Regular noun: only takes singular, adds "s" to form plural.
      regNoun : Str -> Noun =
        \car -> noun car (car + "s") ;
    
      -- Adjective is just a single string.
      adj : Str -> {s : Str} =
        \red -> {s = red} ;
  
      -- Copula inflects in number.
      copula : Number -> Str =
        \n -> case n of {
          Sg => "is" ;
          Pl => "are"
          } ;
    }

Defined FoodEng

### Italian concrete syntax using RGL modules

We use the Resource Grammar Library to hide the linguistic details.

See also Tutorial _[Lesson 4: Using the resource grammar library](https://www.grammaticalframework.org/doc/tutorial/gf-tutorial.html#toc69)_

In [4]:
concrete FoodIta of Food = open SyntaxIta, ParadigmsIta in {
    lincat
      Comment = S ;
      Item = NP ;
      Kind = CN ;
      Quality = AP ;

    lin
      Pred item quality = mkS (mkCl item quality) ;
      This kind = mkNP this_Det kind ;
      That kind = mkNP that_Det kind ;
      These kind = mkNP these_Det kind ;
      Those kind = mkNP those_Det kind ;
      Mod quality kind = mkCN quality kind ;
      Very quality = mkAP very_AdA quality ;
      Wine = mkCN (mkN "vino") ;
      Pizza = mkCN (mkN "pizza") ;
      Cheese = mkCN (mkN "formaggio") ;
      Fish = mkCN (mkN "pesce") ;
      Fresh = mkAP (mkA "fresco") ;
      Warm = mkAP (mkA "caldo") ;
      Italian = mkAP (mkA "italiano") ;
      Expensive = mkAP (mkA "caro") ;
      Delicious = mkAP (mkA "delizioso") ;
      Boring = mkAP (mkA "noioso") ;
}

Defined FoodIta

In [5]:
parse -lang=Eng "this pizza is boring" | linearize -lang=Ita

questa pizza è noiosa


In [6]:
view parse -lang=Eng "this Italian pizza is very expensive" | visualize_parse -lang=Eng

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02M\x00\x00\x01\xbb\x08\x06\x00\x00\x00\xde\xd9\xa1…

In [7]:
view p -lang=Ita "questa pizza italiana è molto cara" | vp -lang=Ita

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x028\x00\x00\x01\xbb\x08\x06\x00\x00\x00\xce\xb9\xef…